In [17]:
import os

os.environ['GROQ_API_KEY'] = '.........'

In [18]:
! git clone https://github.com/gspeter-max/internShalaProject.git

Cloning into 'internShalaProject'...
remote: Enumerating objects: 177, done.
remote: Counting objects: 100% (177/177), done.
remote: Compressing objects: 100% (156/156), done.
remote: Total 177 (delta 43), reused 31 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (177/177), 2.63 MiB | 15.64 MiB/s, done.
Resolving deltas: 100% (43/43), done.


In [19]:
%cd /content/internShalaProject/Yardstick_Assignment

/content/internShalaProject/Yardstick_Assignment


In [20]:
! pwd

/content/internShalaProject/Yardstick_Assignment


In [21]:
! pip install groq

In [22]:
# %load /content/internShalaProject/Yardstick_Assignment/getFunctionCallling.py
import json

def getJsonArg(
        name : str,
        email : str,
        phone : str,
        location : str,
        age : int
    ):

    return_json = json.dumps({
        "name": name,
        "email": email,
        "phone": phone,
        "location": location,
        "age": age
    })

    return return_json

tools  = [
    {
        "type": "function",
        "function": {
            "name": "getJsonArg",
            "description": (
                "Captures and structures a user's complete contact and demographic information. "
                "Use this function when a user needs to be registered, created in a system, or "
                "have their full profile saved. It should only be called after all five details "
                "(name, email, phone, location, age) have been provided."
            ),
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "The full name of the user mentioned in the chat."
                    },
                    "email": {
                        "type": "string",
                        "description": "The email address provided by the user."
                    },
                    "phone": {
                        "type": "string",
                        "description": "The contact or phone number of the user."
                    },
                    "location": {
                        "type": "string",
                        "description": "The city or general location of the user."
                    },
                    "age": {
                        "type": "integer",
                        "description": "The age of the user as an integer."
                    }
                },
                "required": ["name", "email", "phone", "location", "age"]
            }
        }
    }
]

available_function_tool_name = {'getJsonArg': getJsonArg }


In [23]:
# from transformers import  AutoTokenizer
import typing
import json
from groq import Groq
from fastapi import FastAPI
# from dotenv import load_dotenv
from pydantic import BaseModel
# from getFunctionCallling import available_function_tool_name, tools
import os

# app = FastAPI()
class ChatCompletionMessageParam( BaseModel ):
    role : str
    content : typing.Union[dict[str, str], str ]
    GroqApiKey: str



class CreateSummary:
    def __init__(self):
        self.summary = []


# tokenizer =  AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf')
maximum_tokens = 1000
global_token_counter = 0
keep_noOf_message = 20
k_messages_feed_into_llm = 3
max_token_posible_in_output = 1000

if keep_noOf_message < k_messages_feed_into_llm:
    raise RuntimeError(f'keep_noOf_message must be lower than k_messages_feed_into_llm  ( {keep_noOf_message} < {k_messages_feed_into_llm  }) ')

def summarization_logic( summarizationClass : CreateSummary , client ):

    for index, actual_content in enumerate(reversed(summarizationClass.summary)):
        if index >= keep_noOf_message:
            content = summarizationClass.summary[ index: ]
        else:
            content = actual_content

        llm_response = client.chat.completions.create(
            messages =[
                {
                    "role": "user",
                    "content": f"Summarize the following text into the shortest possible version while preserving all essential \
                        information and eliminating redundancy. Ensure no important detail is lost. The result must be concise,\
                             precise, and information-dense. Text: {content}"
                }
            ],
            model = 'gemma2-9b-it'
        )
        response = llm_response.choices[0].message.content
        summarizationClass.summary.append(response)

_summary_class = CreateSummary()

# @app.post('/v2/getResponse')
def get_response(userquery : ChatCompletionMessageParam ):
    global global_token_counter
    global _summary_class
    try:

        client = Groq( api_key = userquery.GroqApiKey )
        # input_tokens = tokenizer(userquery.content).input_ids
        input_tokens = userquery.content

        if (len(input_tokens) >= maximum_tokens):
            return {"error": f"input tokens >= {maximum_tokens}"}
        else:
            global_token_counter += len(input_tokens)

        if  ((global_token_counter + len(input_tokens)) >= maximum_tokens):
            summarization_logic( summarizationClass= _summary_class , client = client)

        if len(_summary_class.summary) <= k_messages_feed_into_llm:
            content = f"past summary of conversation : {_summary_class.summary} \n\n" + f"current user_qestion : {userquery.content}"
        else:
            content = f"past summary of conversation : {_summary_class.summary[:k_messages_feed_into_llm]} \n\n" + f"current user_qestion : {userquery.content}"


        llm_input_message = [
                {
                    "role": userquery.role,
                    "content" : content
                }
            ]
        response = client.chat.completions.create(
            messages=llm_input_message,
            model = 'gemma2-9b-it'
        ).choices[0].message.content

        # if len(tokenizer.tokenize(response)) >= max_token_posible_in_output:
        if len(response) >= max_token_posible_in_output:

            _o5th_path = int(len(response) / 5)
            return_response = f'{response[:_o5th_path]} </---------/>  {response[-_o5th_path :]}'
        else:
            return_response = response

        _summary_class.summary.append(f'userquery : {userquery.content}  llmRespose : {response}')
        return {'response' : return_response }

    except Exception as e:
        return {"error": f"got error during getting response from llm due to : {e}"}

# @app.post('/v2/getJsonResponse')
def get_json_response(userquery : ChatCompletionMessageParam ):
    try:

        client = Groq( api_key = userquery.GroqApiKey )

        llm_input_message = [
                {
                    "role": userquery.role,
                    "content" : userquery.content
                }
            ]
        response = client.chat.completions.create(
            messages=llm_input_message,
            model = 'gemma2-9b-it',
            tools= tools
        )

        # for now ,we are assume we have only one tool
        for tool_metadata in response.choices[0].message.tool_calls:

            tool_args_in_str  = tool_metadata.function.arguments
            tool_args_in_json = json.loads(tool_args_in_str)

            function_name = tool_metadata.function.name

            function_object = available_function_tool_name[function_name]
            function_response = function_object(**tool_args_in_json)

            # if userquery.care_about_task2 is True:
            return {"args_for_next_function": function_response }

        #     json_format_function_response = json.dumps({
        #         "tool_call_id": tool_metadata.id,
        #         "role": "tool",
        #         "content": function_response
        #     })

        #     llm_input_message.append( json.loads(json_format_function_response) )

        # final_response = client.chat.completions.create(
        #     messages= llm_input_message,
        #     model= 'gemma2-9b-it'
        # ).choices[0].message.content

        # return {'json_response': final_response}
    except Exception as e:
        return {'error': f"{e}"}


In [24]:
args = ChatCompletionMessageParam(
    role = 'user',
    content = 'my name is peter and my age is 19 and location is  abohar and phone number is 7717565570 email is gspeter67@gmail.com ',
    GroqApiKey = os.getenv('GROQ_API_KEY')
    )

print(get_json_response(args))
print(get_response(args))

{'args_for_next_function': '{"name": "peter", "email": "gspeter67@gmail.com", "phone": "7717565570", "location": "abohar", "age": 19}'}
{'response': "Please remember that sharing personal information like your phone number and email address online can be risky. It's important to protect your privacy. \n\nI understand you're telling me your name is Peter, you're 19 years old, you live in Abohar, and you gave me your email address and phone number.\n\nIs there anything else I can help you with? Perhaps you'd like to discuss something specific about Abohar, or maybe you have a question about something else entirely?  \n\n"}
